In [41]:
import sys
sys.path.append("../../")

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from utils.ncbi.names import split_bio, map_and_add_tax_ids, standardize_core, generate_names_df, names_db_path, find_tax_id
from typing import Tuple
from utils.data_paths import *

taxonomy_dict = {"s": "species", "g": "genus", "f": "family", "o": "order", "c": "class", "p": "phylum", "k": "kingdom"}

In [42]:
replacement_dict = {"Clostridium_clostridioforme": "Clostridium_clostridiiforme"}

split_name_replacement_dict = {
    "LACHNOSPIRACEAE_UNCLASSIFIED": "UNCLASSIFIED_LACHNOSPIRACEAE",
    "CLOSTRIDIALES_FAMILY_XIII-INCERTAE-SEDIS-UNCLASSIFIED":"CLOSTRIDIALES_FAMILY_XIII--INCERTAE-SEDIS",
    "ACTINOMYCES_ODONTOLYTICUS": "SCHAALIA_ODONTOLYTICA",
}

In [43]:
# We want a csv file with genus, relative abundance.
def clean_biobakery_with_taxid(df, rank="g") -> Tuple[pd.DataFrame, pd.DataFrame]:
    data = df.copy()

    # Get the indices so we can split then. They are of the form ex: ...|g__Bacteroides|s__Bacteroides_vulgatus
    indices = data.index.to_list()
    # Tax_ids are of the form #|#|# ...
    tax_ids = data["TAX_ID"].to_list()

    # This makes a list of lists, where each sublist is the taxonomy split by the rank.
    splitted = [i.split("|") for i in indices]
    splitted_ids = [i.split("|") for i in tax_ids]

    new_index = []
    new_ids = []
    # Traverse all of the rows.
    for c, i in enumerate(splitted):
        # Traverse the sublists.
        for c2, j in enumerate(i):
            # If it matches the rank we want, then append it to the new index.
            if j.startswith(f"{rank}__"):
                new_index.append(j.replace(f"{rank}__", ""))
                # The taxid is the same index as the taxonomy.
                new_ids.append(splitted_ids[c][c2])

    # Set the new index.
    data.index = new_index

    # Make a new dataframe with the new index and the new_ids. We will use this to merge since we do not want to sum the tax_ids.
    taxid_df = pd.DataFrame(index=new_index, data=new_ids, columns=["TAX_ID"])

    # Sum the rows where the genus/species is the same.
    grouped = data.groupby(data.index).sum(numeric_only=True)

    # Now, we want to rename the columns to be the sample names, since they are of the form sampleid_###.
    columns = grouped.columns.to_list()
    new_cols = [i.split("_")[0] for i in columns]
    grouped.columns = new_cols

    # Divide all the values by 100.
    grouped = grouped / 100

    # We should now remove duplicates in the taxIDs (since genera can be equivalent). 
    # If there are duplicates, then we should make it equal to the first value.
    taxid_df = taxid_df[~taxid_df.index.duplicated(keep="first")]

    return grouped, taxid_df

# We want to save to csv, but we want a csv for each column.
def save_to_csv(df, taxid_df, output_path, rank="s"):
    # Get the columns.
    columns = df.columns.to_list()

    # Iterate over the columns.
    for c, i in enumerate(columns):
        # Get the column.
        col = df[[i]]

        # Join the tax_ids to the column.
        # Now, we want to add the new_ids to the dataframe by joining on the index.
        col = col.join(taxid_df, how="left")

        # TODO: Figure out why I did this in the first place, removing for now.
        # We need to multiply each column by the reciprocal of the minimum non-zero value.

        # # Get the minimum non-zero value.
        # minimum = col[col > 0].min().min()
        # col[i] = col[i] / minimum

        # # Round to closest integer.
        # col[i] = col[i].round()

        # display(col.head())

        # Save to csv.
        col.to_csv(os.path.join(output_path, f"{i.upper()}_{taxonomy_dict[rank]}_relabund_annotated.csv"), index_label=f"{taxonomy_dict[rank]}")

In [44]:
# In their great wisdom, they decided that the taxid should not be incldued in the file. 
# Therefore, we will need to standardize the names and attach the taxids.
def clean_biobakery_merged(df: pd.DataFrame, rank="g"):
    """
    Cleans the merged biobakery file (after cutting into species_relab.txt) and attached TAX_IDs.
    """
    data = df.copy()

    # Get the indices so we can split then. They are of the form ex: ...|g__Bacteroides|s__Bacteroides_vulgatus
    indices = data.index.to_list()

    # This makes a list of lists, where each sublist is the taxonomy split by the rank.
    splitted = [i.split("|") for i in indices]
    # print("splitted: ", splitted)

    new_index = []
    # Traverse all of the rows.
    for c, i in enumerate(splitted):
        # Traverse the sublists.
        for c2, j in enumerate(i):
            # If it matches the rank we want, then append it to the new index.
            if j.startswith(f"{rank}__"):
                new_index.append(j.replace(f"{rank}__", ""))

    # print(new_index)
    # Set the new index.
    data.index = new_index

    # Sum the rows where the genus/species is the same.
    grouped = data.groupby(data.index).sum(numeric_only=True)

    # Now, we want to rename the columns to be the sample names, since they are of the form sampleid_###.
    columns = grouped.columns.to_list()
    new_cols = [i.split("_")[0] for i in columns]
    grouped.columns = new_cols

    # Divide all the values by 100.
    grouped = grouped / 100

    return grouped

In [45]:
def standardize_merged(df: pd.DataFrame):
    """ 
    This function takes the merged dataframe (converted to RA with clean_biobakery_merged) \  
    and standardizes the names plus adds the taxids.
    """
    names_df = generate_names_df(names_db_path, load_pickle=True)

    # Get the index. Replace names with those in replacement_dict.
    index = df.index.to_list()
    index = [replacement_dict[i] if i in replacement_dict else i for i in index]
    df.index = index

    split_names = split_bio(df)

    standard_df = standardize_core(input_df=df, split_names=split_names)

    # Replace the split_name using the replacement_dict.
    standard_df.set_index("split_name", inplace=True)

    # Replace any split_names that are in the replacement_dict.
    standard_df.index = [split_name_replacement_dict[i] if i in split_name_replacement_dict else i for i in standard_df.index]

    annotated = map_and_add_tax_ids(df=standard_df, names_df=names_df)

    # We want to split the tax_id column off the dataframe.
    taxid_df = annotated[["tax_id"]]

    # If any rows contain nan, then we raise an error.
    if taxid_df.isna().any().any():
        # Print the rows that contain nan.
        display(taxid_df[taxid_df.isna().any(axis=1)])
        raise ValueError("There are nan values in the tax_id column.")

    annotated.drop(columns=["tax_id"], inplace=True)

    return annotated, taxid_df

### Main Code
Run the below function for cleaning of biobakery. Change rank for the desired rank.

In [46]:
# data_path = hmpTongue.biobakery4
# # data_path = "pipelines/bmock12/biobakery4/species_relab.txt"
# output_dir = "pipelines/hmp/tongue/bio4/"
# if not os.path.exists(data_path):
#     raise Exception("Data file does not exist!")
# # data_path = "pipelines/bmock12/biobakery4/species_relab.txt"
# data = pd.read_csv(data_path, sep="\t", index_col=0)
# # display(data.head())
# # print(data.shape)

In [47]:
# rank = "s"
# names, taxids = clean_biobakery_with_taxid(data, rank=rank)
# save_to_csv(names, taxids, output_dir, rank=rank)

In [48]:
dpths = make_data_list()

# We have an issue with the HMP. Let's make a replacement dictionary to fix this for now.
hmp_dict = {
    "Clostridiales_unclassified": "Unclassified",
    "Firmicutes_unclassified": "Unclassified",
    "Lachnospiraceae_unclassified": "Unclassified",
    "Ruminococcaceae_unclassified": "Unclassified",
    # "Ruminococcaceae_unclassified": "Unclassified",
}

amos_dict = {
    "Prevotella_copri_clade_A": "Prevotella_copri",
}

def main(data_path: str, output_dir: str, rank: str, replace_dict: dict = None) -> None:
    print(data_path)
    data = pd.read_csv(data_path, sep="\t", index_col=0)
    display(data)

    cleaned = clean_biobakery_merged(data, rank=rank)
        
    if replace_dict is not None:
        cleaned.rename(index=replace_dict, inplace=True)

    annotated, ids = standardize_merged(cleaned)

    display(annotated)

    save_to_csv(annotated, ids, output_path=output_dir, rank=rank)

def clean_all():
    for i in dpths:
        output_path = os.path.join(i.path, "bio4")
        if not os.path.exists(output_path):
            output_path = os.path.join(i.path, "biobakery4")
            os.makedirs(output_path, exist_ok=True)

        print(output_path)
        if i.biobakery4 == "":
            continue
        
        if "hmp" in i.path:
            replace_dict = hmp_dict
        elif "mixed" in i.path or "hilo" in i.path:
            replace_dict = amos_dict
        else:
            replace_dict = None

        main(i.biobakery4, output_dir=output_path, rank="g", replace_dict=replace_dict)
        main(i.biobakery4, output_dir=output_path, rank="s", replace_dict=replace_dict)

    for i in dpths:
        output_path = os.path.join(i.path, "bio3")
        if not os.path.exists(output_path):
            output_path = os.path.join(i.path, "biobakery3")
            os.makedirs(output_path, exist_ok=True)

        print(output_path)
        if i.biobakery3 == "":
            continue
        
        if "hmp" in i.path:
            replace_dict = hmp_dict
        elif "mixed" in i.path or "hilo" in i.path:
            replace_dict = amos_dict
        else:
            replace_dict = None

        main(i.biobakery3, output_dir=output_path, rank="g", replace_dict=replace_dict)
        main(i.biobakery3, output_dir=output_path, rank="s", replace_dict=replace_dict)

clean_all()

# def clean_eg():
#     # We also have to add the bio_eg.
#     bio_eg_path = "/Volumes/TBHD_share/valencia/pipelines/NIST/pipelines/bio4_eg/metaphlan/merged/species_relab.txt"
#     output_eg = "../../pipelines/nist/bio4/"

#     main(bio_eg_path, output_eg, rank="g")
#     main(bio_eg_path, output_eg, rank="s")

# clean_eg()

/Users/valenciaem/coding/pipelines/pipelines/bmock12/biobakery4
/Volumes/TBHD_share/valencia/pipelines/bmock12/biobakery4/metaphlan/merged/species_relab.txt


,sub_SRR8073716_R1_taxonomic_profile,sub_SRR8073716_R2_taxonomic_profile
# taxonomy,,
k__Bacteria|p__Bacteroidetes|c__Flavobacteriia|o__Flavobacteriales|f__Flavobacteriaceae|g__Muricauda|s__Muricauda_sp_ES_050,24.31063,25.05761
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Oceanospirillales|f__Halomonadaceae|g__Halomonas|s__Halomonas_sp_HL_93,41.56277,41.15733
k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rhodobacterales|f__Rhodobacteraceae|g__Thioclava|s__Thioclava_sp_ES_032,3.13392,3.11096
k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Propionibacteriales|f__Propionibacteriaceae|g__Ponticoccus|s__Ponticoccus_gilvus,1.18807,1.16572
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Alteromonadales|f__Alteromonadaceae|g__Marinobacter|s__Marinobacter_sp_AC_23,0.00007,0.00027
k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rhizobiales|f__Cohaesibacteraceae|g__Cohaesibacter|s__Cohaesibacter_sp_ES_047,6.41304,6.35279
k__Bacteria|p__Bacteroidetes|c__Flavobacteriia|o__Flavobacteriales|f__Flavobacteriaceae|g__Muricauda|s__Muricauda_sp_ES_050|t__SGB2690,24.31063,25.05761
k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Micromonosporales|f__Micromonosporaceae|g__Micromonospora|s__Micromonospora_echinofusca,0.68733,0.67860
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Oceanospirillales|f__Halomonadaceae|g__Halomonas|s__Halomonas_sp_HL_93|t__SGB11831,41.56277,41.15733


The pkl file was last modified (and hopefully generated) on 2022-10-27 14:39:35+00:00


,sub,sub
COHAESIBACTER,0.128261,0.127056
HALOMONAS,0.831255,0.823147
MARINOBACTER,0.443272,0.438727
MICROMONOSPORA,0.024560,0.024385
MURICAUDA,0.486213,0.501152
PONTICOCCUS,0.023761,0.023314
THIOCLAVA,0.062678,0.062219


/Volumes/TBHD_share/valencia/pipelines/bmock12/biobakery4/metaphlan/merged/species_relab.txt


...

,Mix-A,Mix-B,Mix-C,Mix-D
ACHROMOBACTER,0.000000,0.032408,0.267875,0.000000
ACINETOBACTER,0.007697,0.098990,0.000000,0.000000
AEROMONAS,0.002409,0.003222,0.002241,0.001718
ENTEROCOCCUS,0.000098,0.000000,0.045564,0.361074
ESCHERICHIA,0.345128,0.044368,0.294877,0.034862
KLEBSIELLA,0.025510,0.403768,0.000000,0.000000
LEGIONELLA,0.005567,0.009223,0.006493,0.006262
LISTERIA,0.000000,0.000000,0.028280,0.220235
NEISSERIA,0.297135,0.000000,0.000000,0.025508
SALMONELLA,0.024478,0.367777,0.000000,0.000000


/Volumes/TBHD_share/valencia/pipelines/NIST/pipelines/bio3/metaphlan/merged/species_relab.txt


,Mix-A_S2_L001_taxonomic_profile,Mix-B_S3_L001_taxonomic_profile,Mix-C_S4_L001_taxonomic_profile,Mix-D_S5_L001_taxonomic_profile
# taxonomy,,,,
k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Listeriaceae|g__Listeria|s__Listeria_monocytogenes,0.00000,0.00000,2.82804,22.02346
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Vibrionales|f__Vibrionaceae|g__Vibrio|s__Vibrio_furnissii,0.00000,0.00000,3.77734,32.44526
k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Staphylococcaceae|g__Staphylococcus|s__Staphylococcus_aureus,28.95041,0.00000,0.00000,2.52811
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Enterococcus|s__Enterococcus_faecalis,0.00978,0.00000,4.55638,36.10740
k__Bacteria|p__Firmicutes|c__Bacilli|o__Bacillales|f__Staphylococcaceae|g__Staphylococcus|s__Staphylococcus_argenteus,0.24741,0.00000,0.00000,0.06070
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Streptococcaceae|g__Streptococcus|s__Streptococcus_pyogenes,0.00000,4.02455,31.68969,0.00000
k__Bacteria|p__Proteobacteria|c__Betaproteobacteria|o__Neisseriales|f__Neisseriaceae|g__Neisseria|s__Neisseria_meningitidis,29.71352,0.00000,0.00000,2.55085
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Aeromonadales|f__Aeromonadaceae|g__Aeromonas|s__Aeromonas_hydrophila,0.24087,0.32224,0.22407,0.17182
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Legionellales|f__Legionellaceae|g__Legionella|s__Legionella_pneumophila,0.55674,0.92229,0.64932,0.62621


The pkl file was last modified (and hopefully generated) on 2022-10-27 14:39:35+00:00


,Mix-A,Mix-B,Mix-C,Mix-D
ACHROMOBACTER_DENITRIFICANS,0.000000,0.000000,0.001378,0.000000
ACHROMOBACTER_RUHLANDII,0.000000,0.002979,0.024312,0.000000
ACHROMOBACTER_XYLOSOXIDANS,0.000000,0.029429,0.242185,0.000000
ACINETOBACTER_BAUMANNII,0.005891,0.076085,0.000000,0.000000
ACINETOBACTER_NOSOCOMIALIS,0.000000,0.001432,0.000000,0.000000
ACINETOBACTER_PITTII,0.001806,0.021473,0.000000,0.000000
AEROMONAS_HYDROPHILA,0.002409,0.003222,0.002241,0.001718
ENTEROCOCCUS_FAECALIS,0.000098,0.000000,0.045564,0.361074
ESCHERICHIA_COLI,0.345128,0.044368,0.294877,0.034862
KLEBSIELLA_PNEUMONIAE,0.021326,0.315497,0.000000,0.000000
